In [1]:
from __future__ import print_function
import tensorflow as tf
from keras.datasets import cifar100
import keras as k


import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import print_summary, to_categorical
from keras import backend as K
import sys
import os
import numpy as np

Using TensorFlow backend.


In [2]:

NUM_CLASSES = 100


In [3]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [4]:
print("X train shape: {}".format(x_train.shape))
print("y train shape: {}".format(y_train.shape))
print("X test shape: {}".format(x_test.shape))
print("Y test shape: {}".format(y_test.shape))

X train shape: (50000, 32, 32, 3)
y train shape: (50000, 1)
X test shape: (10000, 32, 32, 3)
Y test shape: (10000, 1)


In [5]:
import matplotlib.pyplot as plt
index = 50
plt.imshow(x_train[index,:])
print(y_train[index,:])

[73]


In [6]:
#one hot encoding
y_train_ = k.utils.to_categorical(y_train, num_classes=100)
y_test_ = k.utils.to_categorical(y_test, num_classes=100)


In [7]:
#Normalizing the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train_ = x_train/255.0
x_test_ = x_test/255.0

In [8]:
model = Sequential()

#stack 1... input = 32,32,3
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', 
                 kernel_regularizer = l2(0.01), input_shape = (32,32,3)))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
#... output = 32,32,64
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
#... output = 16,16,64


#stack 2
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
#... output = 16,16,128
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))
#... output = 8,8,128

#stack 3
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = 'same', activation = 'relu',
                 kernel_regularizer = l2(0.01)))
#... output = 8,8,512
model.add(MaxPooling2D(pool_size = (2,2), strides = 1))
#... output = 7,7,512

model.add(Flatten()) #7*7*512 = 25088
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

BATCH_SIZE = 32

model.fit(x_train_, y_train, epochs = 5, steps_per_epoch=50000//BATCH_SIZE, shuffle=True)

Epoch 1/5
